# VGG16 Setup for Pig Disease Classification

accuracy with all data: 0.34700122475624084
accuracy with 10% of data: 

In [69]:

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout


In [70]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4008110205203455991
xla_global_id: -1
]


In [ ]:
import random
import shutil

data_dir = 'data/'

sampled_data_dir = 'sampled_data/'  # Directory for sampled data

# Create sampled data directory
if not os.path.exists(sampled_data_dir):
    os.makedirs(sampled_data_dir)

# Randomly sample 10% of data
for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path):
        sampled_class_path = os.path.join(sampled_data_dir, class_name)
        os.makedirs(sampled_class_path, exist_ok=True)
        
        # Get all files in this class and sample 10%
        files = os.listdir(class_path)
        # sampled_files = random.sample(files, max(1, len(files) // 20))  # Sample 20%

        # sampled_files = random.sample(files, max(1, len(files) // 10))  # Sample 10%
        sampled_files = files # load 100%
        # Copy sampled files to the new directory
        for file in sampled_files:
            src = os.path.join(class_path, file)
            dst = os.path.join(sampled_class_path, file)
            shutil.copy(src, dst)

print(f"Sampled data is stored in: {sampled_data_dir}")

Sampled data is stored in: sampled_data/


In [72]:

# Define paths
data_dir = 'data/sampled_data'

# Data preprocessing
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2  # 80-20 split for the sampled data
)

train_gen = datagen.flow_from_directory(
    sampled_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    sampled_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 19608 images belonging to 10 classes.
Found 4902 images belonging to 10 classes.


In [73]:

# Load VGG16 without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False


In [74]:

# Add custom layers on top of the base model
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(train_gen.class_indices), activation='softmax')(x)  # Number of classes

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [75]:

# Train the model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5
)


Epoch 1/5
613/613 ━━━━━━━━━━━━━━━━━━━━ 1241s 2s/step - accuracy: 0.1707 - loss: 2.2989 - val_accuracy: 0.2052 - val_loss: 2.0591
Epoch 2/5
613/613 ━━━━━━━━━━━━━━━━━━━━ 1251s 2s/step - accuracy: 0.2176 - loss: 1.9989 - val_accuracy: 0.2544 - val_loss: 2.0018
Epoch 3/5
613/613 ━━━━━━━━━━━━━━━━━━━━ 1239s 2s/step - accuracy: 0.2292 - loss: 1.9614 - val_accuracy: 0.2801 - val_loss: 1.9912
Epoch 4/5
613/613 ━━━━━━━━━━━━━━━━━━━━ 1262s 2s/step - accuracy: 0.2375 - loss: 1.9408 - val_accuracy: 0.2876 - val_loss: 1.9700
Epoch 5/5
613/613 ━━━━━━━━━━━━━━━━━━━━ 1279s 2s/step - accuracy: 0.2507 - loss: 1.9089 - val_accuracy: 0.3072 - val_loss: 1.9590


In [81]:

# Evaluate the model
test_loss, test_acc = model.evaluate(val_gen)
print(f"Test accuracy: {test_acc}")


154/154 ━━━━━━━━━━━━━━━━━━━━ 240s 2s/step - accuracy: 0.3976 - loss: 1.8572
Test accuracy: 0.3802529573440552


In [77]:

# Save the model for future use
model.save('vgg16_pig_disease_model.h5')


In [78]:
from tensorflow.keras.models import load_model

# Load the model
# model = load_model('vgg16_pig_disease_model.h5')

# Confirm the model is loaded
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,352,736 (92.90 MB)

 Trainable params: 3,212,682 (12.26 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 6,425,366 (24.51 MB)

In [79]:

# Fine-tune the model (optional)
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Recompile with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train again
history_finetune = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5
)


Epoch 1/5
613/613 ━━━━━━━━━━━━━━━━━━━━ 1375s 2s/step - accuracy: 0.2437 - loss: 1.8935 - val_accuracy: 0.3429 - val_loss: 1.9182
Epoch 2/5
613/613 ━━━━━━━━━━━━━━━━━━━━ 1383s 2s/step - accuracy: 0.2673 - loss: 1.8587 - val_accuracy: 0.3454 - val_loss: 1.8985
Epoch 3/5
613/613 ━━━━━━━━━━━━━━━━━━━━ 1410s 2s/step - accuracy: 0.2772 - loss: 1.8416 - val_accuracy: 0.3372 - val_loss: 1.9100
Epoch 4/5
613/613 ━━━━━━━━━━━━━━━━━━━━ 1405s 2s/step - accuracy: 0.2850 - loss: 1.8158 - val_accuracy: 0.3727 - val_loss: 1.8774
Epoch 5/5
613/613 ━━━━━━━━━━━━━━━━━━━━ 1404s 2s/step - accuracy: 0.3063 - loss: 1.7876 - val_accuracy: 0.3803 - val_loss: 1.8696


In [88]:

# Use the model for predictions on new images
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np


# Path to the test image
img_path = 'C:/Users/Gebruiker/Desktop/School/Minor/PigAI/implementation5/testimage/l (11).jpg'

# Load and preprocess the image
img = load_img(img_path, target_size=(224, 224))
img_array = img_to_array(img) / 255.0
img_array = img_array.reshape(1, 224, 224, 3)

# Make the prediction
prediction = model.predict(img_array)

# Get the class indices from the training generator
class_indices = train_gen.class_indices  
class_labels = {v: k for k, v in class_indices.items()}  # Reverse the mapping

# Display each class with its predicted probability
for i, prob in enumerate(prediction[0]):
    class_name = class_labels[i]  # Get the class name
    print(f"Class: {class_name} | Probability: {prob:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Class: Healthy | Probability: 0.3808
Class: Infected_Bacterial_Erysipelas | Probability: 0.0064
Class: Infected_Bacterial_Greasy_Pig_Disease | Probability: 0.1673
Class: Infected_Environmental_Dermatitis | Probability: 0.0000
Class: Infected_Environmental_Sunburn | Probability: 0.4242
Class: Infected_Fungal_Pityriasis_Rosea | Probability: 0.0001
Class: Infected_Fungal_Ringworm | Probability: 0.0000
Class: Infected_Parasitic_Mange | Probability: 0.0186
Class: Infected_Viral_Foot_and_Mouth_Disease | Probability: 0.0023
Class: Infected_Viral_Swinepox | Probability: 0.0004
